In [1]:
#执行这个文件必须在python 3.8版本下
import os
import numpy as np
import pickle
from active_utils.file_tools import save_obj,load_obj
from active_utils.dataset_process_tools import DatasetInfo,split_train_test_sets
import random

    


## 加载EXACT09和LIDC数据集

In [3]:
LidcInfo = DatasetInfo("/mnt/wangc/LIDC/Precrop_dataset_for_LIDC-IDRI_128", "lidc", 128)
LidcInfo.get_case_names("/mnt/wangc/LIDC", "lidc")

Exact09Info = DatasetInfo(
    "/mnt/wangc/EXACT09/Precrop_dataset_for_EXACT09_128", "exact09", 128
)
Exact09Info.get_case_names("/mnt/wangc/EXACT09/EXACT09_3D", "exact09")

In [4]:

# create_data_dict会比较慢，大约1分钟，也可以用之前保存的字典
LidcInfo.create_data_dict()
Exact09Info.create_data_dict()

# Exact09Info.data_dict=load_obj("/home/wangc/now/NaviAirway/saved_objs/for_128_objs/exact_128_info.pkl")
# LidcInfo.data_dict=load_obj("/home/wangc/now/NaviAirway/saved_objs/for_128_objs/lidc_128_info")


## 将数据集分为 训练集/测试集

In [5]:
names = np.concatenate((Exact09Info.processed_names,LidcInfo.processed_names))
# names = np.concatenate((EXACT09_names,LIDC_IDRI_names))

# you can split train/test by yourself
# just show an example

test_names = ['LIDC_IDRI_0066', 'LIDC_IDRI_0328', 'LIDC_IDRI_0376',
        'LIDC_IDRI_0441', 'EXACT09_CASE13', 'LIDC_IDRI_0744',
        'EXACT09_CASE08', 'EXACT09_CASE01', 'EXACT09_CASE05',
        'LIDC_IDRI_1004']
# test_names = [ 'EXACT09_CASE03',
#         'EXACT09_CASE04',]
# print("test name: "+str(test_names))
train_names = []
for name in names:
    if name not in test_names:
        train_names.append(name)
train_names=np.array(train_names)
# print("train names: "+str(train_names))
print(len(train_names),len(test_names))

116 10


In [6]:
train_test_set_dict_128 = split_train_test_sets(Exact09Info.data_dict, LidcInfo.data_dict, train_names, test_names)
data_dict_org_128 = train_test_set_dict_128["train"]

In [ ]:
# train_test_set_dict_256 = split_train_test_sets(data_dict_EXACT09_256, data_dict_LIDC_IDRI_256, train_names, test_names)
# data_dict_org_256 = train_test_set_dict_256["train"]

# train_test_set_dict_128 = split_train_test_sets(data_dict_EXACT09_128, data_dict_LIDC_IDRI_128, train_names, test_names)
# data_dict_org_128 = train_test_set_dict_128["train"]

## 获取训练集info

### 之前保存过的一些info

In [10]:
# save_obj(data_dict_org,'/home/wangc/now/NaviAirway/saved_objs/for_128_objs/data_dict_org')

data_dict_org=load_obj('/home/wangc/now/pure/saved_objs/for_128_objs/training_info/data_dict_org.pkl')
len(data_dict_org)

# data_dict_only_negtive=load_obj('/home/wangc/now/NaviAirway/saved_objs/for_128_objs/data_dict_only_negtive')
# len(data_dict_only_negtive)


25024

### 随机提取部分数据集

In [8]:
import random


def extract_random_num_percent_by_key(num,dictionary):#随机抽取字典中num个样本
    keys = list(dictionary.keys())
    random.shuffle(keys)
    randon_dict={key: dictionary[key] for key in keys}

    num_elements = len(keys)
    num_to_extract = int(num_elements * num)
    top_num_percent_keys = keys[:num_to_extract]
    return {key: randon_dict[key] for key in top_num_percent_keys}


In [13]:
# data_dict_20=extract_random_num_percent_by_key(0.2,data_dict_org)
data_dict_40=extract_random_num_percent_by_key(0.4,data_dict_org)
# data_dict_60=extract_random_num_percent_by_key(0.6,data_dict_org)
# save_obj(data_dict_20, "/home/wangc/now/NaviAirway/saved_objs/for_128_objs/training_info_0119/random_info_30")
save_obj(data_dict_40, "/home/wangc/now/pure/saved_objs/for_128_objs/training_info_1214/random_info_40")
# save_obj(data_dict_60, "/home/wangc/now/NaviAirway/saved_objs/for_128_objs/training_info_0119/random_info_60")


SyntaxError: invalid character in identifier (95201820.py, line 2)

## 基于uncertainy的训练数据获取

In [7]:
#总样本数num1=比例*len(data_dict_org)
#从negtive抽取等比例的样本放到result_dict
#根据org和negtive的len计算num1
#取uncertainy前num个样本放到dict2
#使用save_obj保存

import random

def select_from_uncertainy(uncertainy_path,data_dict1,data_dict2,num,save_path=None):
    #对于negtive,抽取m%的样本
    def extract_random_num_percent_by_key(num,dictionary):#随机抽取字典中num个样本
        keys = list(dictionary.keys())
        random.shuffle(keys)
        randon_dict={key: dictionary[key] for key in keys}

        num_elements = len(keys)
        num_to_extract = int(num_elements * num)
        top_num_percent_keys = keys[:num_to_extract]
        return {key: randon_dict[key] for key in top_num_percent_keys}
    result_dict=extract_random_num_percent_by_key(num,data_dict2)
    #计算从uncertainy抽取的样本数num3
    num1=int(num*len(data_dict_org))
    num2=len(result_dict)
    num3=num1-num2

    #读取uncertainy_dict
    with open(uncertainy_path, 'rb') as file:
        loaded_data = pickle.load(file)
        uncertainy_dict = loaded_data['uncertainy_dict']
    print(len(uncertainy_dict))
    #将uncertainy升序排序
    sorted_dict = dict(sorted(uncertainy_dict.items(), key=lambda item: item[1]))
    sorted_list = list(sorted_dict.keys())

    #抽取前sample_number个样本并放入result_dict
    al_list = sorted_list[:num3]
    for i in al_list:
        print(i)
        temp = i[:-7]
        result_dict[temp] = data_dict1[temp]

    #使用save_obj函数保存
    if save_path is not None:
        save_obj(result_dict, save_path)
    return num1,num2,num3


In [ ]:


data_dict_org=load_obj('/home/wangc/now/NaviAirway/saved_objs/for_128_objs/data_dict_org.pkl')
data_dict_only_negtive=load_obj('/home/wangc/now/NaviAirway/saved_objs/for_128_objs/data_dict_only_negtive')
uncertainy_path = '/home/wangc/now/NaviAirway/saved_var/uncertainy_128_data.pkl'

num=0.6
save_path=f'/home/wangc/now/NaviAirway/saved_objs/for_128_objs/training_info_0119/al_info_{int(num*100)}1'
# # select_from_uncertainy(uncertainy_path,data_dict_org,data_dict_only_negtive,num,save_path)
# num1,num2,num3=select_from_uncertainy(uncertainy_path,data_dict_org,data_dict_only_negtive,num,save_path=save_path)
# num1,num2,num3
select_from_uncertainy(uncertainy_path,data_dict_org,data_dict_only_negtive,num,save_path=save_path)

In [ ]:


data_dict_org=load_obj('/home/wangc/now/NaviAirway/saved_objs/for_128_objs/data_dict_org.pkl')
data_dict_only_negtive=load_obj('/home/wangc/now/NaviAirway/saved_objs/for_128_objs/data_dict_only_negtive')
ae_uncertainy_path = '/home/wangc/now/NaviAirway/saved_var/ae1_uncertainy.pkl'

num=0.6
save_path=f'/home/wangc/now/NaviAirway/saved_objs/for_128_objs/training_info_0119/ae1_info_{int(num*100)}'
# select_from_uncertainy(uncertainy_path,data_dict_org,data_dict_only_negtive,num,save_path)
num1,num2,num3=select_from_uncertainy(ae_uncertainy_path ,data_dict_org,data_dict_only_negtive,num,save_path=save_path)
num1,num2,num3

In [21]:
data_dict70=load_obj('/home/wangc/now/NaviAirway/saved_objs/for_128_objs/training_info/random_info_703.pkl')
data_dict100=load_obj('/home/wangc/now/NaviAirway/saved_objs/for_128_objs/training_info/random_info_1002.pkl')
data_dict100.keys()
flag=True
for key in data_dict70.keys():
    if key not in data_dict100.keys():
        flag=False
print(flag)
print(len(data_dict70),len(data_dict100))

True
17516 25024
